# Knowledge based Movie Recomendation System

In [1]:
!python --version

Python 3.9.12


In [2]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

In [3]:
data = pd.read_csv("movies.csv")
data

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,407045464.0,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,475766228.0,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
4,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,269000000.0,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722709,888716,Assassin Legend,Action,zh,NaN,0.600,NaN,2018-04-13,0.0,0.0,75.0,Released,NaN,0.000,0.0,Tse Miu-William Liao-Yuri Liu-Zhang Chunzhong-...,NaN,/i3uYuF47mlMPWEK4IYIA4dVYm5t.jpg,/5YjksQqNm5qvuYv7mJzel2C9qla.jpg,NaN
722710,872952,The Anatomy of a Virus: The Making of Antiviral,NaN,en,A behind-the-scenes documentary filmed during ...,0.600,NaN,NaN,0.0,0.0,30.0,Released,NaN,0.000,0.0,Brandon Cronenberg-Caleb Landry Jones-Sarah Ga...,NaN,/7wexHEkawISnNMd8MLhQ6Nx28Ra.jpg,NaN,NaN
722711,294742,Gospođa- Slobodan,NaN,en,Short video spot.,0.600,NaN,NaN,0.0,0.0,5.0,Released,NaN,0.000,0.0,NaN,NaN,NaN,NaN,NaN
722712,506737,Revolt of the Empire of the Apes,Action-Adventure,en,Earth has become the Apes playground and human...,0.600,NaN,2017-07-19,0.0,0.0,75.0,Released,NaN,2.000,1.0,James Carolus-Marie DeLorenzo-Steve Diasparra-...,NaN,/pP1svh3fZVUvu41ptuvXkBr2mBG.jpg,NaN,NaN


## Analysing Data

In [4]:
print(data.shape)

(722714, 20)


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722714 entries, 0 to 722713
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    722714 non-null  int64  
 1   title                 722710 non-null  object 
 2   genres                511967 non-null  object 
 3   original_language     722714 non-null  object 
 4   overview              604177 non-null  object 
 5   popularity            722714 non-null  float64
 6   production_companies  337202 non-null  object 
 7   release_date          670472 non-null  object 
 8   budget                722714 non-null  float64
 9   revenue               722714 non-null  float64
 10  runtime               688281 non-null  float64
 11  status                722714 non-null  object 
 12  tagline               108165 non-null  object 
 13  vote_average          722714 non-null  float64
 14  vote_count            722714 non-null  float64
 15  

In [6]:
data.isnull().sum()

id                           0
title                        4
genres                  210747
original_language            0
overview                118537
popularity                   0
production_companies    385512
release_date             52242
budget                       0
revenue                      0
runtime                  34433
status                       0
tagline                 614549
vote_average                 0
vote_count                   0
credits                 225030
keywords                512372
poster_path             185079
backdrop_path           500051
recommendations         687932
dtype: int64

In [7]:
data.duplicated().sum()

0

## Cleaning Data

In [8]:
# dropping unnecesary title 
df = data.drop(["production_companies", "popularity", "budget", "revenue", "status", "recommendations", "runtime", "vote_average", "backdrop_path", "tagline"], axis=1)

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.title.duplicated().sum()

86992

In [11]:
df[["title", "release_date"]].duplicated().sum()

2235

In [12]:
df.drop_duplicates(subset=["title","release_date"], inplace=True)

In [13]:
df = df[df.vote_count >= 350].reset_index()

In [14]:
df.isnull().sum()

index                  0
id                     0
title                  0
genres                 0
original_language      0
overview               1
release_date           0
vote_count             0
credits                8
keywords             238
poster_path            0
dtype: int64

In [15]:
df.fillna("", inplace = True)

In [16]:
index = df[(df.genres == "") & (df.overview == "")].index
df.drop(index, inplace=True)

In [17]:
df.genres = df.genres.apply(lambda x: " ".join(x.split("-")))
df.keywords = df.keywords.apply(lambda x: " ".join(x.split("-")))
df.credits = df.credits.apply(lambda x: " ".join(x.replace(" ", "").split("-")[:5]))

In [18]:
# making tags consisting of overview, genre, credits, keywords and original language
df["tags"] =df.overview + " "+ df.genres + " "  +df.credits + " " +df.keywords + " " + df.original_language

In [19]:
# making new framework with important features
new_df = df[["id", "title", "tags", 'poster_path']]

In [20]:
new_df.tags = new_df.tags.apply(lambda x:x.lower())

In [21]:
new_df.head()

,id,title,tags,poster_path
0,615656,Meg 2: The Trench,an exploratory dive into the deepest depths of...,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg
1,758323,The Pope's Exorcist,father gabriele amorth chief exorcist of the v...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg
2,667538,Transformers: Rise of the Beasts,when a new threat capable of destroying the en...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg
3,640146,Ant-Man and the Wasp: Quantumania,super-hero partners scott lang and hope van dy...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg
4,677179,Creed III,after dominating the boxing world adonis creed...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg


### Stremming

Stemming is the process of reducing words to their base or root form.

In [22]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [23]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)    

In [24]:
new_df["tags"] = new_df["tags"].apply(stem)

### Text Vectorization

Text vectorization is the process of converting text data into numerical vectors so that they can be used as input for machine learning models.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",max_features=2000)

In [26]:
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [27]:
cv.get_feature_names()[80:85]

['alter', 'altern', 'alway', 'amazon', 'ambit']

## ML Modeling

Cosine Similarity is a process of creating a machine learning model that utilizes the cosine similarity metric to determine the similarity between two pieces of text.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [29]:
similarity.shape

(7659, 7659)

## Testing

In [30]:
def recommend(movies):
    movie_index = new_df[new_df.title == movies].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [31]:
recommend("Toy Story")

Toy Story 4
Toy Story 2
Small Soldiers
The Indian in the Cupboard
Toy Story 3


In [32]:
recommend("The Conjuring")

The Conjuring: The Devil Made Me Do It
Grave Encounters
Terrified
A Haunted House 2
The Conjuring 2


__Observation__ : The function is returning 5 similar movies.